# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [53]:
# Importing the dataset
pokemon = pd.read_csv("./Pokemon.csv")
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [54]:
# your code here

verified_total = pokemon['HP'].sum() + pokemon['Attack'].sum() + pokemon['Defense'].sum() +  pokemon['Sp. Atk'].sum() + pokemon['Sp. Def'].sum() + pokemon['Speed'].sum()

verified_total == pokemon['Total'].sum()

True

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [4]:
list(pokemon.columns)

['#',
 'Name',
 'Type 1',
 'Type 2',
 'Total',
 'HP',
 'Attack',
 'Defense',
 'Sp. Atk',
 'Sp. Def',
 'Speed',
 'Generation',
 'Legendary']

In [12]:
# your code here

#Create dummy columns for all the types and drop not necessary columns 
new_pokemon = pd.get_dummies(pokemon, columns = ['Type 1']).drop(['Type 2','#','Name','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation','Legendary'], axis = 1)
new_pokemon.head()

,Total,Type 1_Bug,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,Type 1_Ice,Type 1_Normal,Type 1_Poison,Type 1_Psychic,Type 1_Rock,Type 1_Steel,Type 1_Water
0,318,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,405,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,525,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,625,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,309,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [13]:
#Create dummy columns for all the types and drop not necessary columns 
new_pokemon2 = pd.get_dummies(pokemon, columns = ['Type 2']).drop(['Type 1','#','Name','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation','Legendary'], axis = 1)
new_pokemon2.head()

,Total,Type 2_Bug,Type 2_Dark,Type 2_Dragon,Type 2_Electric,Type 2_Fairy,Type 2_Fighting,Type 2_Fire,Type 2_Flying,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,318,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,405,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,525,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,625,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,309,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
Name_columns = ['Total',
 'Bug',
 'Dark',
 'Dragon',
 'Electric',
 'Fairy',
 'Fighting',
 'Fire',
 'Flying',
 'Ghost',
 'Grass',
 'Ground',
 'Ice',
 'Normal',
 'Poison',
 'Psychic',
 'Rock',
 'Steel',
 'Water']

In [18]:
new_pokemon.columns = Name_columns
new_pokemon2.columns = Name_columns

In [17]:
list(new_pokemon.columns)

['Total',
 'Bug',
 'Dark',
 'Dragon',
 'Electric',
 'Fairy',
 'Fighting',
 'Fire',
 'Flying',
 'Ghost',
 'Grass',
 'Ground',
 'Ice',
 'Normal',
 'Poison',
 'Psychic',
 'Rock',
 'Steel',
 'Water']

In [19]:
list(new_pokemon2.columns)

['Total',
 'Bug',
 'Dark',
 'Dragon',
 'Electric',
 'Fairy',
 'Fighting',
 'Fire',
 'Flying',
 'Ghost',
 'Grass',
 'Ground',
 'Ice',
 'Normal',
 'Poison',
 'Psychic',
 'Rock',
 'Steel',
 'Water']

In [21]:
new_pokemon.head()

,Total,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,318,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,405,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,525,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,625,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,309,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [22]:
new_pokemon2.head()

,Total,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,318,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,405,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,525,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,625,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,309,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
final_pokemon = new_pokemon + new_pokemon2
final_pokemon.head()

,Total,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,636,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,810,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,1050,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,1250,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,618,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [29]:
final_pokemon = final_pokemon.drop(columns = ['Total'])
final_pokemon.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [64]:
#to join with the initial dataframe by the index

pokemon_all = pokemon.join(final_pokemon)
corr_pokemon = pokemon_all.corr()

In [63]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Grass-Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Grass-Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Grass-Poison
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Grass-Poison
4,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,Fire


In [65]:
# your code here
corr_pokemon.sort_values('Total', ascending=False)

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
Total,0.119813,1.000000,0.618748,0.736211,0.612787,0.747250,0.717609,0.575943,0.048384,0.501758,...,0.003641,-0.052592,0.015060,0.060248,-0.105331,-0.090441,0.124688,0.032731,0.109703,-0.021665
Sp. Atk,0.088759,0.747250,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018,0.036437,0.448907,...,0.035184,0.001312,-0.107346,0.070236,-0.184013,-0.043722,0.237125,-0.106214,-0.005608,0.025885
Attack,0.102298,0.736211,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240,0.051451,0.345408,...,-0.015735,-0.062674,0.127416,0.021007,-0.074769,-0.073370,-0.069773,0.101043,0.107505,-0.070578
Sp. Def,0.085817,0.717609,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133,0.028486,0.363937,...,0.044885,-0.005243,-0.076820,0.060370,-0.112124,-0.046722,0.190286,0.019179,0.103424,-0.024765
HP,0.097614,0.618748,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952,0.058683,0.273620,...,-0.062262,-0.046120,0.074916,0.082021,0.111999,-0.075668,0.027239,-0.035514,-0.043037,0.029906
Defense,0.094786,0.612787,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227,0.042419,0.246377,...,0.060864,-0.006948,0.134451,0.020174,-0.174609,-0.093942,0.010685,0.298242,0.350559,0.006259
Speed,0.010733,0.575943,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000,-0.023121,0.326715,...,-0.060599,-0.095578,-0.095803,-0.005371,0.052224,-0.030134,0.108020,-0.165331,-0.098291,-0.049030
Legendary,0.153396,0.501758,0.273620,0.345408,0.246377,0.448907,0.363937,0.326715,0.079794,1.000000,...,-0.014494,-0.066736,-0.007329,0.041136,-0.086246,-0.086195,0.169224,-0.012571,0.019437,-0.065781
Dragon,0.141996,0.229705,0.138023,0.212999,0.105864,0.194388,0.139235,0.123244,0.102059,0.225625,...,-0.019410,-0.078818,0.052430,0.015173,-0.098702,-0.055525,-0.026557,-0.032360,-0.044417,-0.083285
Psychic,0.003396,0.124688,0.027239,-0.069773,0.010685,0.237125,0.190286,0.108020,-0.021772,0.169224,...,-0.070947,-0.094009,-0.079079,-0.042311,-0.112381,-0.103195,1.000000,-0.069031,0.024540,-0.099639


In [ ]:
#The number one pokemon type is 'Sp. Atk'

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [57]:
# your code here

pokemon[['Type 2']] = pokemon[['Type 2']].fillna(" ")
def pokemon_type(row):
    type_1 = row['Type 1']
    type_2 = row['Type 2']
    
    if type_2 == " ":  
        return type_1
    else:
        return str(type_1) + "-" + str(type_2)

pokemon['Combo Type'] = pokemon.apply(pokemon_type, axis = 1)

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Grass-Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Grass-Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Grass-Poison
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Grass-Poison
4,4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,Fire


In [59]:
new_pokemon3 = pd.get_dummies(pokemon, columns = ['Combo Type']).drop(['Type 1','Type 2','#','Name','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation','Legendary'], axis = 1)
new_pokemon3.head()

,Total,Combo Type_Bug,Combo Type_Bug-Electric,Combo Type_Bug-Fighting,Combo Type_Bug-Fire,Combo Type_Bug-Flying,Combo Type_Bug-Ghost,Combo Type_Bug-Grass,Combo Type_Bug-Ground,Combo Type_Bug-Poison,...,Combo Type_Water-Fighting,Combo Type_Water-Flying,Combo Type_Water-Ghost,Combo Type_Water-Grass,Combo Type_Water-Ground,Combo Type_Water-Ice,Combo Type_Water-Poison,Combo Type_Water-Psychic,Combo Type_Water-Rock,Combo Type_Water-Steel
0,318,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,525,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,625,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,309,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
corr_pokemon2 = new_pokemon3.corr()
corr_pokemon2.sort_values('Total', ascending=False)

,Total,Combo Type_Bug,Combo Type_Bug-Electric,Combo Type_Bug-Fighting,Combo Type_Bug-Fire,Combo Type_Bug-Flying,Combo Type_Bug-Ghost,Combo Type_Bug-Grass,Combo Type_Bug-Ground,Combo Type_Bug-Poison,...,Combo Type_Water-Fighting,Combo Type_Water-Flying,Combo Type_Water-Ghost,Combo Type_Water-Grass,Combo Type_Water-Ground,Combo Type_Water-Ice,Combo Type_Water-Poison,Combo Type_Water-Psychic,Combo Type_Water-Rock,Combo Type_Water-Steel
Total,1.000000,-0.178699,-0.016537,0.047979,0.008309,-0.017369,-0.058753,-0.037054,-0.037625,-0.089742,...,0.062210,-0.024374,-0.011526,-0.045257,-0.001128,0.039182,-0.004317,0.030361,-0.003756,0.028003
Combo Type_Dragon-Flying,0.149777,-0.012809,-0.004352,-0.004352,-0.004352,-0.011602,-0.003075,-0.007557,-0.004352,-0.010727,...,-0.005333,-0.008167,-0.004352,-0.005333,-0.009780,-0.005333,-0.005333,-0.006894,-0.006162,-0.003075
Combo Type_Dragon-Ice,0.128737,-0.009040,-0.003071,-0.003071,-0.003071,-0.008188,-0.002170,-0.005333,-0.003071,-0.007571,...,-0.003764,-0.005764,-0.003071,-0.003764,-0.006903,-0.003764,-0.003764,-0.004866,-0.004349,-0.002170
Combo Type_Dragon-Psychic,0.127066,-0.010445,-0.003549,-0.003549,-0.003549,-0.009461,-0.002508,-0.006162,-0.003549,-0.008748,...,-0.004349,-0.006660,-0.003549,-0.004349,-0.007976,-0.004349,-0.004349,-0.005622,-0.005025,-0.002508
Combo Type_Psychic-Fighting,0.104173,-0.009040,-0.003071,-0.003071,-0.003071,-0.008188,-0.002170,-0.005333,-0.003071,-0.007571,...,-0.003764,-0.005764,-0.003071,-0.003764,-0.006903,-0.003764,-0.003764,-0.004866,-0.004349,-0.002170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Combo Type_Poison,-0.064380,-0.020368,-0.006920,-0.006920,-0.006920,-0.018449,-0.004890,-0.012016,-0.006920,-0.017058,...,-0.008481,-0.012987,-0.006920,-0.008481,-0.015552,-0.008481,-0.008481,-0.010963,-0.009799,-0.004890
Combo Type_Normal-Fairy,-0.069525,-0.011685,-0.003970,-0.003970,-0.003970,-0.010584,-0.002806,-0.006894,-0.003970,-0.009787,...,-0.004866,-0.007451,-0.003970,-0.004866,-0.008923,-0.004866,-0.004866,-0.006289,-0.005622,-0.002806
Combo Type_Bug-Poison,-0.089742,-0.018183,-0.006178,-0.006178,-0.006178,-0.016469,-0.004366,-0.010727,-0.006178,1.000000,...,-0.007571,-0.011594,-0.006178,-0.007571,-0.013884,-0.007571,-0.007571,-0.009787,-0.008748,-0.004366
Combo Type_Normal-Flying,-0.092626,-0.025913,-0.008804,-0.008804,-0.008804,-0.023471,-0.006222,-0.015288,-0.008804,-0.021702,...,-0.010790,-0.016523,-0.008804,-0.010790,-0.019786,-0.010790,-0.010790,-0.013947,-0.012467,-0.006222


In [ ]:
#The number one pokemon combo type is 'Dragon-Flying'